## NetGO2.0 dataset
https://deepgo.cbrc.kaust.edu.sa/data/deepgozero/data-netgo.tar.gz

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
train_terms = pd.read_csv("../../input/cafa-5-protein-function-prediction/Train/train_terms.tsv",sep="\t")
train_terms.set_index('EntryID', inplace=True)
train_terms.head()

,term,aspect
EntryID,,
A0A009IHW8,GO:0008152,BPO
A0A009IHW8,GO:0034655,BPO
A0A009IHW8,GO:0072523,BPO
A0A009IHW8,GO:0044270,BPO
A0A009IHW8,GO:0006753,BPO


In [4]:
unique_train_ids = train_terms.index.unique()

In [7]:
train_id_terms = train_terms.loc[unique_train_ids[1]]

In [8]:
train_id_terms

,term,aspect
EntryID,,
A0A021WW32,GO:0048869,BPO
A0A021WW32,GO:0048856,BPO
A0A021WW32,GO:0022008,BPO
A0A021WW32,GO:0065007,BPO
A0A021WW32,GO:0007275,BPO
...,...,...
A0A021WW32,GO:0005721,CCO
A0A021WW32,GO:0070013,CCO
A0A021WW32,GO:0043227,CCO


In [13]:
netgo_train_terms = pd.read_csv("../../input/data-netgo/train.txt",
                                sep="\t", names=["EntryID", "term", "aspect", "unknown"])
netgo_aspect_map = {'cc' : 'CCO', 'bp' : 'BPO', 'mf' : 'MFO'}
netgo_train_terms['aspect'] = netgo_train_terms['aspect'].apply(lambda x : netgo_aspect_map[x])
netgo_train_terms.set_index('EntryID', inplace=True)
netgo_train_terms.head()

,term,aspect,unknown
EntryID,,,
Q5BHH6,GO:0005576,CCO,227321
C8VRD6,GO:0005622,CCO,227321
Q5BHC4,GO:0016567,BPO,227321
Q5BH26,GO:0000124,CCO,227321
C8VUY4,GO:0005622,CCO,227321


In [14]:
netgo_valid_terms = pd.read_csv("../../input/data-netgo/valid.txt",
                                sep="\t", names=["EntryID", "term", "aspect", "unknown"])
netgo_valid_terms['aspect'] = netgo_valid_terms['aspect'].apply(lambda x : netgo_aspect_map[x])
netgo_valid_terms.set_index('EntryID', inplace=True)
netgo_valid_terms.head()

,term,aspect,unknown
EntryID,,,
P13021,GO:0005515,MFO,44689
P27133,GO:0030036,BPO,44689
P27133,GO:0061803,CCO,44689
Q9XYS8,GO:0001845,BPO,44689
Q9XYS8,GO:0006911,BPO,44689


In [15]:
netgo_test_terms = pd.read_csv("../../input/data-netgo/test.txt",
                                sep="\t", names=["EntryID", "term", "aspect", "unknown"])
netgo_test_terms['aspect'] = netgo_test_terms['aspect'].apply(lambda x : netgo_aspect_map[x])
netgo_test_terms.set_index('EntryID', inplace=True)
netgo_test_terms.head()

,term,aspect,unknown
EntryID,,,
P13021,GO:0051015,MFO,44689
P27133,GO:0005515,MFO,44689
P27133,GO:0015629,CCO,44689
P27133,GO:0043621,MFO,44689
P27133,GO:0001891,CCO,44689


In [18]:
netgo_terms = pd.concat([netgo_train_terms, netgo_valid_terms, netgo_test_terms])
netgo_terms = netgo_terms.drop(columns='unknown')
netgo_terms.head()

,term,aspect
EntryID,,
Q5BHH6,GO:0005576,CCO
C8VRD6,GO:0005622,CCO
Q5BHC4,GO:0016567,BPO
Q5BH26,GO:0000124,CCO
C8VUY4,GO:0005622,CCO


In [20]:
len(netgo_terms), len(train_terms)

(673339, 5363863)

In [21]:
common_vals_counts = train_terms.eq(netgo_terms).sum()
common_vals_counts

term        606882
aspect    34541316
dtype: int64

In [22]:
df_unique = pd.concat([netgo_terms, train_terms]).drop_duplicates(keep=False)
df_unique

,term,aspect
EntryID,,
P18695,GO:0000091,BPO
Q5BEU9,GO:0000169,BPO
C8VT14,GO:0000168,BPO
Q4WLW8,GO:1900778,BPO
A0A1D8PPK6,GO:0000093,BPO
...,...,...
Q9Z4P6,GO:0050485,MFO
R9QMY8,GO:0102701,MFO
X2EVV9,GO:0102626,MFO


## Add these to terms

In [59]:
import json
precalc_fn = "../../output/precalculated_terms.json"

with open(precalc_fn, "r") as outfile:
    precalculated_terms = json.load(outfile)

In [60]:
len(precalculated_terms.keys())

142246

In [61]:
unique_netgo_ids = netgo_terms.index.unique()

In [62]:
unique_netgo_ids

Index(['Q5BHH6', 'C8VRD6', 'Q5BHC4', 'Q5BH26', 'C8VUY4', 'Q5BGS1', 'C8VUB6',
       'Q5BGN3', 'Q5BG90', 'Q5BG78',
       ...
       'V9HW37', 'V9HW56', 'V9HWA0', 'V9HWD0', 'V9HWF5', 'V9HWH0', 'V9HWJ5',
       'W5RWE1', 'W5XKT8', 'X5D778'],
      dtype='object', name='EntryID', length=127819)

In [63]:
num_new_terms = np.sum([1 for term in unique_netgo_ids if term not in precalculated_terms])
new_netgo_terms = [term for term in unique_netgo_ids if term not in precalculated_terms]
num_new_terms

4442

In [29]:
new_terms

['C8VRD6',
 'C8VU63',
 'C8VSZ3',
 'C8VMZ5',
 'C8VLW7',
 'C8VNW8',
 'C8VTL8',
 'C8VAB7',
 'C8VM21',
 'Q5BCY8',
 'Q5BCY5',
 'Q5BCY4',
 'C8VKI3',
 'C8VKU5',
 'Q5BB72',
 'C8VN54',
 'Q5B643',
 'C8VEY3',
 'C8VGK0',
 'Q5B1V0',
 'Q5B0M3',
 'C8V3A5',
 'Q5AZF0',
 'Q5AZD4',
 'Q5AY77',
 'C8VBX3',
 'Q5ASB5',
 'Q5ARJ7',
 'Q5ARA0',
 'Q5AS02',
 'Q5AT91',
 'Q9Y8G1',
 'P0C0V3',
 'Q5AZ36',
 'Q5AZ35',
 'Q5BDF1',
 'Q5B4E7',
 'Q5B4I0',
 'C8VMA8',
 'C8VJE7',
 'C8V0H6',
 'C8VQ65',
 'P87208',
 'Q5BC29',
 'Q5AXK2',
 'C8VFD4',
 'G5EAZ9',
 'G5EAT8',
 'G5EAX7',
 'Q4WJL0',
 'Q4WTC9',
 'Q4WSY6',
 'Q4WS94',
 'Q4WS30',
 'Q4WRP3',
 'Q4WRA6',
 'Q4WHS9',
 'Q4WHL9',
 'Q4X1F3',
 'Q4X1B0',
 'Q4X168',
 'Q4X0G7',
 'Q4X083',
 'Q4WZY9',
 'Q4WFX1',
 'Q4WXF1',
 'Q4WXR1',
 'Q4WXU3',
 'Q4W997',
 'Q4WPF0',
 'Q4WPY2',
 'Q4WQF0',
 'Q4WE02',
 'Q4WE16',
 'Q4WEM3',
 'Q4WEV5',
 'Q4WU85',
 'Q4WUP8',
 'Q4WV27',
 'Q4WVA8',
 'Q4WVP5',
 'Q4WW75',
 'Q4WD10',
 'Q4WDM0',
 'Q4WLX3',
 'Q4WLR7',
 'Q4WLJ3',
 'Q4WL91',
 'Q4WA81',
 'Q4WGP3',
 'Q4WGZ5',

In [48]:
def netgo_terms_for_train_id(train_id:str):
    train_id_terms = netgo_terms[netgo_terms.index == train_id]
    #train_id_terms = netgo_terms.loc[train_id]
    return {
        'BPO' : train_id_terms[train_id_terms.aspect == 'BPO']['term'].values.tolist(),
        'CCO' : train_id_terms[train_id_terms.aspect == 'CCO']['term'].values.tolist(),
        'MFO' : train_id_terms[train_id_terms.aspect == 'MFO']['term'].values.tolist(),
    }

netgo_terms_for_train_id('C8VRD6')

{'BPO': [], 'CCO': ['GO:0005622'], 'MFO': []}

In [50]:
for i, term in enumerate(unique_netgo_ids):
    if i % 100 == 0: print(i)
    
    new_terms = netgo_terms_for_train_id(term)
    if term in precalculated_terms:
        # update
        for ont in ['BPO', 'CCO', 'MFO']:
            precalculated_terms[term][ont] = np.unique(precalculated_terms[term][ont] + new_terms[ont]).tolist()
    else:
        precalculated_terms[term] = new_terms

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [51]:
precalc2_fn = "../../output/precalculated_terms_netgo.json"
with open(precalc2_fn, "w") as outfile:
    json.dump(precalculated_terms, outfile)

## Prepare fasta file only from new unseen seqs

In [53]:
pip install fastaparser

/bin/bash: /home/joni/miniconda3/envs/cafa5/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [54]:
import fastaparser

In [68]:
new_netgo_fasta_seqs = []
added_ids = []

for fasta in [
    '../../input/data-netgo/bp/train_data.fa',
    '../../input/data-netgo/bp/test_data.fa',
    '../../input/data-netgo/bp/valid_data.fa',
    '../../input/data-netgo/cc/train_data.fa',
    '../../input/data-netgo/cc/test_data.fa',
    '../../input/data-netgo/cc/valid_data.fa',
    '../../input/data-netgo/mf/train_data.fa',
    '../../input/data-netgo/mf/valid_data.fa',
    '../../input/data-netgo/mf/test_data.fa',
]: 
    print(fasta)
    with open(fasta) as fasta_file:
        parser = fastaparser.Reader(fasta_file)
        for seq in parser:
            if seq.id in new_netgo_terms and seq.id not in added_ids:
                added_ids.append(seq.id)
                new_netgo_fasta_seqs.append(seq)

../input/data-netgo/bp/train_data.fa
../input/data-netgo/bp/test_data.fa
../input/data-netgo/bp/valid_data.fa
../input/data-netgo/cc/train_data.fa
../input/data-netgo/cc/test_data.fa
../input/data-netgo/cc/valid_data.fa
../input/data-netgo/mf/train_data.fa
../input/data-netgo/mf/valid_data.fa
../input/data-netgo/mf/test_data.fa


In [69]:
with open('../../output/netgo_new_proteins.fa', 'w') as fasta_file:
    writer = fastaparser.Writer(fasta_file)
    for seq in new_netgo_fasta_seqs:
        writer.writefasta(seq)

## Diamond db

Run diamond notebook before this

In [70]:
diamond_dir = '../../output/diamond'

In [71]:
!../../diamond/diamond makedb \
  --in ../output/netgo_new_proteins.fa \
  --db $diamond_dir/netgo_new_data.dmnd

/bin/bash: /home/joni/miniconda3/envs/cafa5/lib/libtinfo.so.6: no version information available (required by /bin/bash)
diamond v2.1.8.162 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 12
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: ../output/netgo_new_proteins.fa
Opening the database file...  [0.004s]
Loading sequences...  [0.007s]
Masking sequences...  [0.025s]
Writing sequences...  [0.001s]
Hashing sequences...  [0s]
Loading sequences...  [0s]
Writing trailer...  [0s]
Closing the input file...  [0s]
Closing the database file...  [0.014s]

Database sequences  4031
  Database letters  2283969
     Database hash  e04af6c65ecd68188f67b19e9752e7e8
        Total time  0.054000s


In [72]:
# Run blastp
!../../diamond/diamond blastp \
  --more-sensitive \
  -d $diamond_dir/netgo_new_data.dmnd \
  -q "../input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta" \
  --outfmt 6 qseqid sseqid bitscore pident evalue > $diamond_dir/test_diamond_netqo.res

/bin/bash: /home/joni/miniconda3/envs/cafa5/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [74]:
diamond_df = pd.read_csv('../../output/diamond/test_diamond_netqo.res', sep='\t', names=['qsegid', 'sseqid', 'bitscore', 'pident', 'evalue'])
diamond_df['testid'] = diamond_df['qsegid'].apply(lambda x: str(x).split('\\t')[0])
diamond_df = diamond_df.drop(columns='qsegid')
diamond_df = diamond_df[diamond_df.evalue < 0.001]
diamond_df.head()

,sseqid,bitscore,pident,evalue,testid
0,Q4WI29,285.0,67.3,2.280000e-98,Q9CQV8
1,F4IA59,182.0,44.7,5.000000e-58,Q9CQV8
2,Q4WI29,360.0,80.8,1.480000e-127,P62259
3,F4IA59,224.0,50.0,3.380000e-74,P62259
4,Q4WI29,275.0,63.1,2.010000e-94,P68510
